In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
import numpy as np
import pathlib

#depth of per image
IMG_CHANNELS = 3

In [2]:
#download 5-flowers dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

#count photos of dataset
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

print(data_dir)
#data_dir = '/Users/zhuzhirui/.keras/datasets/flower_photos'

3670
/Users/zhuzhirui/.keras/datasets/flower_photos


In [3]:
CLASS_NAMES = [item.numpy().decode("utf-8") for item in
               tf.strings.regex_replace(
                 tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/*"),
                 "/Users/zhuzhirui/.keras/datasets/flower_photos/", "")]
print(CLASS_NAMES)

#save all of elements which have not '.' , ignore all of elements which have '.'
CLASS_NAMES = [item for item in CLASS_NAMES if item.find(".") == -1]
print(CLASS_NAMES)
#CLASS_NAMES = ['roses', 'sunflowers', 'daisy', 'dandelion', 'tulips']

['roses', '.DS_Store', 'sunflowers', 'daisy', 'train_set.csv', 'eval_set.csv', 'dandelion', 'tulips', 'LICENSE.txt']
['roses', 'sunflowers', 'daisy', 'dandelion', 'tulips']


In [4]:
#build shuffled image list and shuffled & corresponding label list
def file_convert_numpylist(is_random=True):
    label_list = []
    roses = [item for item in tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/roses/*")]
    for item in roses:
        label_list.append(1)
    sunflowers = [item for item in tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/sunflowers/*")]
    for item in sunflowers:
        label_list.append(2)
    daisy = [item for item in tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/daisy/*")]
    for item in daisy:
        label_list.append(3)
    dandelion = [item for item in tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/dandelion/*")]
    for item in dandelion:
        label_list.append(4)
    tulips = [item for item in tf.io.gfile.glob("/Users/zhuzhirui/.keras/datasets/flower_photos/tulips/*")]
    for item in tulips:
        label_list.append(5)
    image_list = roses + sunflowers + daisy + dandelion + tulips

    label_list = np.asarray(label_list)
    image_list = np.asarray(image_list)

    if is_random:
        list_index = np.arange(len(image_list))
        np.random.shuffle(list_index)
        label_list = label_list[list_index]
        image_list = image_list[list_index]

    return image_list,label_list

In [5]:
image_list,label_list = file_convert_numpylist(is_random=True)

#divided into training set and evaluation set
train_image_list = image_list[:int(0.9*len(image_list))]
train_label_list = label_list[:int(0.9*len(label_list))]

eval_image_list = image_list[int(0.9*len(image_list)):]
eval_label_list = label_list[int(0.9*len(label_list)):]

In [35]:
#encode int64, bytes, float
def int64_feature(values):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))

def bytes_feature(values):
    #if isinstance(values,type(tf.constant(0))):
        #values = values.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def float_feature(values):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[values]))

def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
        #transform Uint8 Tensor image into string Tensor, and numpy function transform into byte
    )

def image_to_tfexample(image, label):
    feature = {
        'label':int64_feature(label),
        'image':image_feature(image)
    }
    return tf.train.Features(feature=feature)

def convert_TFRFiles(image_list, label_list, TFRecord_filename):
    with tf.io.TFRecordWriter(TFRecord_filename) as writer:
        for i in range(len(image_list)):
            #image = open(image_list[i], 'rb').read()
            image = tf.image.decode_jpeg(tf.io.read_file(image_list[i]))#transform image into Uint8 Tensor
            label = label_list[i]
            example = tf.train.Example(features = image_to_tfexample(image, label))
            writer.write(example.SerializeToString())
            print('writing {} of {}'.format(i+1,len(image_list)))
        print('saved in {}'.format(TFRecord_filename))

In [36]:
#training TFRecord
convert_TFRFiles(train_image_list, train_label_list, TFRecord_filename = 'Training_Set.tfrecord')
#evaluation TFRecord
convert_TFRFiles(eval_image_list, eval_label_list, TFRecord_filename = 'Evaluation_Set.tfrecord')

writing 1 of 3303
writing 2 of 3303
writing 3 of 3303
writing 4 of 3303
writing 5 of 3303
writing 6 of 3303
writing 7 of 3303
writing 8 of 3303
writing 9 of 3303
writing 10 of 3303
writing 11 of 3303
writing 12 of 3303
writing 13 of 3303
writing 14 of 3303
writing 15 of 3303
writing 16 of 3303
writing 17 of 3303
writing 18 of 3303
writing 19 of 3303
writing 20 of 3303
writing 21 of 3303
writing 22 of 3303
writing 23 of 3303
writing 24 of 3303
writing 25 of 3303
writing 26 of 3303
writing 27 of 3303
writing 28 of 3303
writing 29 of 3303
writing 30 of 3303
writing 31 of 3303
writing 32 of 3303
writing 33 of 3303
writing 34 of 3303
writing 35 of 3303
writing 36 of 3303
writing 37 of 3303
writing 38 of 3303
writing 39 of 3303
writing 40 of 3303
writing 41 of 3303
writing 42 of 3303
writing 43 of 3303
writing 44 of 3303
writing 45 of 3303
writing 46 of 3303
writing 47 of 3303
writing 48 of 3303
writing 49 of 3303
writing 50 of 3303
writing 51 of 3303
writing 52 of 3303
writing 53 of 3303
wr

writing 434 of 3303
writing 435 of 3303
writing 436 of 3303
writing 437 of 3303
writing 438 of 3303
writing 439 of 3303
writing 440 of 3303
writing 441 of 3303
writing 442 of 3303
writing 443 of 3303
writing 444 of 3303
writing 445 of 3303
writing 446 of 3303
writing 447 of 3303
writing 448 of 3303
writing 449 of 3303
writing 450 of 3303
writing 451 of 3303
writing 452 of 3303
writing 453 of 3303
writing 454 of 3303
writing 455 of 3303
writing 456 of 3303
writing 457 of 3303
writing 458 of 3303
writing 459 of 3303
writing 460 of 3303
writing 461 of 3303
writing 462 of 3303
writing 463 of 3303
writing 464 of 3303
writing 465 of 3303
writing 466 of 3303
writing 467 of 3303
writing 468 of 3303
writing 469 of 3303
writing 470 of 3303
writing 471 of 3303
writing 472 of 3303
writing 473 of 3303
writing 474 of 3303
writing 475 of 3303
writing 476 of 3303
writing 477 of 3303
writing 478 of 3303
writing 479 of 3303
writing 480 of 3303
writing 481 of 3303
writing 482 of 3303
writing 483 of 3303


writing 857 of 3303
writing 858 of 3303
writing 859 of 3303
writing 860 of 3303
writing 861 of 3303
writing 862 of 3303
writing 863 of 3303
writing 864 of 3303
writing 865 of 3303
writing 866 of 3303
writing 867 of 3303
writing 868 of 3303
writing 869 of 3303
writing 870 of 3303
writing 871 of 3303
writing 872 of 3303
writing 873 of 3303
writing 874 of 3303
writing 875 of 3303
writing 876 of 3303
writing 877 of 3303
writing 878 of 3303
writing 879 of 3303
writing 880 of 3303
writing 881 of 3303
writing 882 of 3303
writing 883 of 3303
writing 884 of 3303
writing 885 of 3303
writing 886 of 3303
writing 887 of 3303
writing 888 of 3303
writing 889 of 3303
writing 890 of 3303
writing 891 of 3303
writing 892 of 3303
writing 893 of 3303
writing 894 of 3303
writing 895 of 3303
writing 896 of 3303
writing 897 of 3303
writing 898 of 3303
writing 899 of 3303
writing 900 of 3303
writing 901 of 3303
writing 902 of 3303
writing 903 of 3303
writing 904 of 3303
writing 905 of 3303
writing 906 of 3303


writing 1291 of 3303
writing 1292 of 3303
writing 1293 of 3303
writing 1294 of 3303
writing 1295 of 3303
writing 1296 of 3303
writing 1297 of 3303
writing 1298 of 3303
writing 1299 of 3303
writing 1300 of 3303
writing 1301 of 3303
writing 1302 of 3303
writing 1303 of 3303
writing 1304 of 3303
writing 1305 of 3303
writing 1306 of 3303
writing 1307 of 3303
writing 1308 of 3303
writing 1309 of 3303
writing 1310 of 3303
writing 1311 of 3303
writing 1312 of 3303
writing 1313 of 3303
writing 1314 of 3303
writing 1315 of 3303
writing 1316 of 3303
writing 1317 of 3303
writing 1318 of 3303
writing 1319 of 3303
writing 1320 of 3303
writing 1321 of 3303
writing 1322 of 3303
writing 1323 of 3303
writing 1324 of 3303
writing 1325 of 3303
writing 1326 of 3303
writing 1327 of 3303
writing 1328 of 3303
writing 1329 of 3303
writing 1330 of 3303
writing 1331 of 3303
writing 1332 of 3303
writing 1333 of 3303
writing 1334 of 3303
writing 1335 of 3303
writing 1336 of 3303
writing 1337 of 3303
writing 1338 

writing 1722 of 3303
writing 1723 of 3303
writing 1724 of 3303
writing 1725 of 3303
writing 1726 of 3303
writing 1727 of 3303
writing 1728 of 3303
writing 1729 of 3303
writing 1730 of 3303
writing 1731 of 3303
writing 1732 of 3303
writing 1733 of 3303
writing 1734 of 3303
writing 1735 of 3303
writing 1736 of 3303
writing 1737 of 3303
writing 1738 of 3303
writing 1739 of 3303
writing 1740 of 3303
writing 1741 of 3303
writing 1742 of 3303
writing 1743 of 3303
writing 1744 of 3303
writing 1745 of 3303
writing 1746 of 3303
writing 1747 of 3303
writing 1748 of 3303
writing 1749 of 3303
writing 1750 of 3303
writing 1751 of 3303
writing 1752 of 3303
writing 1753 of 3303
writing 1754 of 3303
writing 1755 of 3303
writing 1756 of 3303
writing 1757 of 3303
writing 1758 of 3303
writing 1759 of 3303
writing 1760 of 3303
writing 1761 of 3303
writing 1762 of 3303
writing 1763 of 3303
writing 1764 of 3303
writing 1765 of 3303
writing 1766 of 3303
writing 1767 of 3303
writing 1768 of 3303
writing 1769 

writing 2115 of 3303
writing 2116 of 3303
writing 2117 of 3303
writing 2118 of 3303
writing 2119 of 3303
writing 2120 of 3303
writing 2121 of 3303
writing 2122 of 3303
writing 2123 of 3303
writing 2124 of 3303
writing 2125 of 3303
writing 2126 of 3303
writing 2127 of 3303
writing 2128 of 3303
writing 2129 of 3303
writing 2130 of 3303
writing 2131 of 3303
writing 2132 of 3303
writing 2133 of 3303
writing 2134 of 3303
writing 2135 of 3303
writing 2136 of 3303
writing 2137 of 3303
writing 2138 of 3303
writing 2139 of 3303
writing 2140 of 3303
writing 2141 of 3303
writing 2142 of 3303
writing 2143 of 3303
writing 2144 of 3303
writing 2145 of 3303
writing 2146 of 3303
writing 2147 of 3303
writing 2148 of 3303
writing 2149 of 3303
writing 2150 of 3303
writing 2151 of 3303
writing 2152 of 3303
writing 2153 of 3303
writing 2154 of 3303
writing 2155 of 3303
writing 2156 of 3303
writing 2157 of 3303
writing 2158 of 3303
writing 2159 of 3303
writing 2160 of 3303
writing 2161 of 3303
writing 2162 

writing 2522 of 3303
writing 2523 of 3303
writing 2524 of 3303
writing 2525 of 3303
writing 2526 of 3303
writing 2527 of 3303
writing 2528 of 3303
writing 2529 of 3303
writing 2530 of 3303
writing 2531 of 3303
writing 2532 of 3303
writing 2533 of 3303
writing 2534 of 3303
writing 2535 of 3303
writing 2536 of 3303
writing 2537 of 3303
writing 2538 of 3303
writing 2539 of 3303
writing 2540 of 3303
writing 2541 of 3303
writing 2542 of 3303
writing 2543 of 3303
writing 2544 of 3303
writing 2545 of 3303
writing 2546 of 3303
writing 2547 of 3303
writing 2548 of 3303
writing 2549 of 3303
writing 2550 of 3303
writing 2551 of 3303
writing 2552 of 3303
writing 2553 of 3303
writing 2554 of 3303
writing 2555 of 3303
writing 2556 of 3303
writing 2557 of 3303
writing 2558 of 3303
writing 2559 of 3303
writing 2560 of 3303
writing 2561 of 3303
writing 2562 of 3303
writing 2563 of 3303
writing 2564 of 3303
writing 2565 of 3303
writing 2566 of 3303
writing 2567 of 3303
writing 2568 of 3303
writing 2569 

writing 2957 of 3303
writing 2958 of 3303
writing 2959 of 3303
writing 2960 of 3303
writing 2961 of 3303
writing 2962 of 3303
writing 2963 of 3303
writing 2964 of 3303
writing 2965 of 3303
writing 2966 of 3303
writing 2967 of 3303
writing 2968 of 3303
writing 2969 of 3303
writing 2970 of 3303
writing 2971 of 3303
writing 2972 of 3303
writing 2973 of 3303
writing 2974 of 3303
writing 2975 of 3303
writing 2976 of 3303
writing 2977 of 3303
writing 2978 of 3303
writing 2979 of 3303
writing 2980 of 3303
writing 2981 of 3303
writing 2982 of 3303
writing 2983 of 3303
writing 2984 of 3303
writing 2985 of 3303
writing 2986 of 3303
writing 2987 of 3303
writing 2988 of 3303
writing 2989 of 3303
writing 2990 of 3303
writing 2991 of 3303
writing 2992 of 3303
writing 2993 of 3303
writing 2994 of 3303
writing 2995 of 3303
writing 2996 of 3303
writing 2997 of 3303
writing 2998 of 3303
writing 2999 of 3303
writing 3000 of 3303
writing 3001 of 3303
writing 3002 of 3303
writing 3003 of 3303
writing 3004 

writing 84 of 367
writing 85 of 367
writing 86 of 367
writing 87 of 367
writing 88 of 367
writing 89 of 367
writing 90 of 367
writing 91 of 367
writing 92 of 367
writing 93 of 367
writing 94 of 367
writing 95 of 367
writing 96 of 367
writing 97 of 367
writing 98 of 367
writing 99 of 367
writing 100 of 367
writing 101 of 367
writing 102 of 367
writing 103 of 367
writing 104 of 367
writing 105 of 367
writing 106 of 367
writing 107 of 367
writing 108 of 367
writing 109 of 367
writing 110 of 367
writing 111 of 367
writing 112 of 367
writing 113 of 367
writing 114 of 367
writing 115 of 367
writing 116 of 367
writing 117 of 367
writing 118 of 367
writing 119 of 367
writing 120 of 367
writing 121 of 367
writing 122 of 367
writing 123 of 367
writing 124 of 367
writing 125 of 367
writing 126 of 367
writing 127 of 367
writing 128 of 367
writing 129 of 367
writing 130 of 367
writing 131 of 367
writing 132 of 367
writing 133 of 367
writing 134 of 367
writing 135 of 367
writing 136 of 367
writing 1